In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) \
           Chrome/39.0.2171.95 Safari/537.36'}

## Creating a list of Supreme Court Opinions and their URLs
I found that caselaw has a repository of Supreme Court ruling opinions. This notebook contains the code I wrote to compile the list of these URLs and titles to use to grab the opinions.

If you want the data I used for this project, you can start by running these cells. 

In [2]:
root_url = "http://caselaw.findlaw.com/court/us-supreme-court/years/"

In [3]:
years = [root_url + str(year) for year in range(1760,2018)]


def Beautiful_soup_grabber(link):
    response = requests.get(link, headers = headers)
    return BeautifulSoup(response.text, "lxml")

def year_getter(years):
    
    y = {}
    for year in years:
        soup = Beautiful_soup_grabber(year)
        souplist = soup.findAll("a")
        for i in souplist:
            if re.search("us-supreme-court", str(i)) and not re.search("years", str(i)) and not re.search("/court/", str(i)):
                b = i["href"]
                y[b] = [re.sub("[^0-9]", "", b.split("/")[-1])]
    return pd.DataFrame(y).transpose().reset_index()

In [4]:
df = year_getter(years)
df.columns = ["case_url", "docket"]

In [5]:
df.head(5)

,case_url,docket
0,http://caselaw.findlaw.com/us-supreme-court/05...,051101
1,http://caselaw.findlaw.com/us-supreme-court/06...,0611951
2,http://caselaw.findlaw.com/us-supreme-court/06...,06263
3,http://caselaw.findlaw.com/us-supreme-court/06...,065590
4,http://caselaw.findlaw.com/us-supreme-court/07...,071390


In [8]:
df.ix[0, "case_url"]

'http://caselaw.findlaw.com/us-supreme-court/05-1101.html'

In [10]:
df.to_pickle("supcourt_yearlist.pickle")

In [9]:
df.shape

(23310, 2)